# My Take for *Ten Essays ON FIZZ BUZZ*

- toc: true
- author: Martin Pan
- branch: master
- badges: true
- comments: true

In [1]:
import itertools
import math
import random
import re
from typing import Callable, Iterator, NamedTuple, List

import numpy as np
import torch
from torch.utils.data import DataLoader


## Introduction

> Write a short program that prints each number from 1 to 100 on a new line.
  For each multiple of 3, print "Fizz" instead of the number.
  For each multiple of 5, print "Buzz" instead of the number.
  For numbers which are multiples of both 3 and 5, print "FizzBuzz" instead of the number.

You probably have seen this problem many times before. This is the "Fizz Buzz" problem sometimes shows up in a code interview. Joel Grus wrote an intriguing and insightful book [*Ten Essays ON FIZZ BUZZ*](https://www.google.com/search?q=ten+essays+on+fizz+buzz&rlz=1C5CHFA_enUS901US902&oq=Ten+Ess&aqs=chrome.0.69i59j0i512j69i57j69i59j0i512j69i61j69i60l2.2461j0j7&sourceid=chrome&ie=UTF-8) to discuss 10 different solutions for this problem in Python and gives informative discusssions about various aspects of Python, mathematics, and software design.

This blog post is my attempts to explain the 10 solutions in the book.

If you plan to read the book (highly recommended!), I hope this blog can give you another perspective to understand the content in the book. If you don't plan to do so, I hope this blog can at least give some knowledge I learned from the book to you.

Most of the codes for those 10 solutions come from [Joel Gru's repository](https://github.com/joelgrus/fizzbuzz). I make small adjustments to run all those solutions in one notebook. Of course, all errors remain mine.

## Testing

In [2]:
def test_fizzbuzz_20(input: List):
    first_20 = ['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz',
                'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17',
                'fizz', '19', 'buzz']
    assert input == first_20
    print('Tess Passed')


## 100 Print

In [3]:
def fizz_buzz_brute(n: int) -> str:
    fizz_buzz = [
        '1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz',
        'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17',
        'fizz', '19', 'buzz', 'fizz', '22', '23', 'fizz', 'buzz',
        '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz',
        '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41',
        'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49',
        'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz',
        '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz',
        'fizz', '67', '68', 'fizz', 'buzz', '71', 'fizz', '73',
        '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz',
        'fizz', '82', '83', 'fizz', 'buzz', '86', 'fizz', '88',
        '89', 'fizzbuzz', '91', '92', 'fizz', '94', 'buzz',
        'fizz', '97', '98', 'fizz', 'buzz'
    ]
    return fizz_buzz[n-1]


In [4]:
output_brute = [fizz_buzz_brute(i) for i in range(1, 21)]
test_fizzbuzz_20(output_brute)

Tess Passed


## Standard Way

In [5]:
def fizz_buzz_standard(n: int) -> str:
    if n % 15 == 0:
        return 'fizzbuzz'
    elif n % 5 == 0:
        return 'buzz'
    elif n % 3 == 0:
        return 'fizz'
    else:
        return str(n)

In [6]:
output_standard = [fizz_buzz_standard(i) for i in range(1, 21)]
test_fizzbuzz_20(output_standard)

Tess Passed


## Cycle of 15

In [7]:
def fizz_buzz_c15(n: int) -> str:
    CYCLE_OF_15 = ['fizzbuzz', None, None, 'fizz',
                   None, 'buzz', 'fizz', None,
                   None, 'fizz', 'buzz', None,
                   'fizz', None, None]
    return CYCLE_OF_15[n % 15] or str(n)


In [8]:
output_c15 = [fizz_buzz_c15(i) for i in range(1, 21)]
test_fizzbuzz_20(output_c15)

Tess Passed


Let's first look at `CYCLE_OF_15[n%15]`. For this part of the function, if the input `n` is only divisible by 3, it will return `'fizz'`. If the input is only divisible by 5, it will return `'buzz'`. If it is divisible by both 3 and 5, it will return `'fizzbuzz'`. Otherwise, it returns `None`. Consider the following code.

In [9]:
CYCLE_OF_15 = ['fizzbuzz', None, None, 'fizz',
               None, 'buzz', 'fizz', None,
               None, 'fizz', 'buzz', None,
               'fizz', None, None]
for n in [3, 5, 15, 17]:
    print(f'{n} % 15 is:', n % 15, end=' ')
    print(f'CYCLE_OF_15[{n} % 15] is:', CYCLE_OF_15[n % 15])


3 % 15 is: 3 CYCLE_OF_15[3 % 15] is: fizz
5 % 15 is: 5 CYCLE_OF_15[5 % 15] is: buzz
15 % 15 is: 0 CYCLE_OF_15[15 % 15] is: fizzbuzz
17 % 15 is: 2 CYCLE_OF_15[17 % 15] is: None


Now let's look at `CYCLE_OF_15[n % 15] or str(n)`. One interesting feature about Python is that for a statement `a or b`. If a is in `[None, [], '', False]`, `a or b` will be equal to `b`. Otherwise, it returns `a`. Consider the following code.

In [10]:
for the_input in ['fizz', None, [], '', False]:
    the_result = the_input or 'b'
    print(f'{the_input} or b, we get: {the_result}')

fizz or b, we get: fizz
None or b, we get: b
[] or b, we get: b
 or b, we get: b
False or b, we get: b


So the idea of this function simple. If the number is divisible by 3, it will return `'fizz' or str(n)`, which is equal to `'fizz'`. The same logic applies to `'buzz'` and `'fizzbuzz'`. If the number is neither divisible by 3 nor by 5, it will return `None or str(n)`, which equals `str(n)`.

## Euclid's Solution

In [11]:
def fizz_buzz_euclid(n: int) -> str:
    hi, lo = max(n, 15), min(n, 15)

    while hi % lo > 0:
        hi, lo = lo, hi % lo

    return {1: str(n), 3: "fizz", 5: "buzz", 15: "fizzbuzz"}[lo]

In [12]:
output_euclid = [fizz_buzz_euclid(i) for i in range(1, 21)]
test_fizzbuzz_20(output_euclid)

Tess Passed


## Trigonometry

In [13]:
def fizz_buzz_tri(n: int) -> str:
    fizz = 'fizz' * int(math.cos(n * math.tau / 3))
    buzz = 'buzz' * int(math.cos(n * math.tau / 5))
    return (fizz + buzz) or str(n)

In [14]:
output_tri = [fizz_buzz_tri(i) for i in range(1, 21)]
test_fizzbuzz_20(output_tri)

Tess Passed


## A Big Number

In [15]:
big_number = 0x134591c9a2c8e4d1647268b23934591c9a2c8e4d16
chunks = re.findall('(0|10|110|111)', f"{big_number:0>167b}")
def label(chunk: str, n: int) -> str:
    labels = [str(n), '', 'fizz', '', '', '', 'buzz', 'fizzbuzz']
    return labels[int(chunk, 2)]
output_big = [label(chunk, n+1) for n, chunk in enumerate(chunks)]

In [16]:
test_fizzbuzz_20(output_big[:20])

Tess Passed


## Infinite Iterables

**TODO: add links for iterator and generator and proofread contents in this section.**

In [17]:
fizzes = itertools.cycle(['', '', 'fizz'])
buzzes = itertools.cycle(['', '', '', '', 'buzz'])
numbers = itertools.count(1)

fizz_buzzes_iter = ((fizz + buzz) or str(n)
                    for fizz, buzz, n in zip(fizzes, buzzes, numbers))

output_iter = [next(fizz_buzzes_iter) for _ in range(20)]


In [18]:
test_fizzbuzz_20(output_iter)

Tess Passed


The central concept around this solultion is `iterable` and `iterators` in Python. But instead of discussing these two concepts in theory, we will try to understand this solution from a practical perspective. To understand more details related to iterables in Python, I will recommend you to watch this video or read the corresponding chapter in *Ten Essays ON FIZZ BUZZ*.

We use the `itertools` library, which is part of the Python standard library and it was imported at the notebook of this notebook.
The first method we use here is `itertools.cycle()`. If we look at the docstring of this method, we will see:
>  Return elements from the iterables until it is exhausted. Then repeats the sequence indefinitely.

That can be confusing. Let's first look at an example.

In [19]:
i = 0
for item in itertools.cycle(['a', 'b', 'c']):
    if i < 7:
        print(item, end=' ')
        i += 1
    else:
        break

a b c a b c a 

From the code above, we can better understand the sentence from the docstring. First, for the part "elements from the iterables", iterables refers to the list `['a', 'b', 'c']` and elements just refer to each value in the list. Second, "until it is exhausted" refers to the fact that it will keep returning elements from the list `['a', 'b', 'c']` until it runs through all of them, as we see from the output above. Finally, "then repeats the sequence indefinitely" means that it will continues to return elements from the list in that sequence (a, then b, then c) repeatly until we stop iterating from it. In the example above, we break the for loop.

So, `fizzes` from `fizzes = itertools.cycle(['', '', 'fizz'])` will just keep return the '', then '', then 'fizz' and repeats. The same idea applies to `buzzes = itertools.cycle(['', '', '', '', 'buzz'])`.

Now, let's look at `itertools.count(1)`. In short, when you iterate through `itertools.count(1)` in a for loop, it will keep returns a number (starts from 1) that plus one from the previous output indefinitely. Let's see an example.

In [20]:
for i in itertools.count(1):
    if i < 10:
        print(i, end= ' ')
    else:
        break

1 2 3 4 5 6 7 8 9 

Note that where the sequence starts depends on what you put inside `itertools.count()`. If you do `itertools.count(-1)`, it will start from -1.

Now let's look at `zip`, which allows us to iterate over multiple iterables in Python. Let's consider an example. Notice how i, j, k corresponds to each list inside `zip`.

In [21]:
for i, j, k in zip(['i1', 'i2'], ['j1', 'j2'], ['k1', 'k2']):
    print(i, j, k)

i1 j1 k1
i2 j2 k2


With that, we can try to understand the behavior in the code `((fizz + buzz) or str(n) for fizz, buzz, n in zip(fizzes, buzzes, numbers))`. Let's consider the output for the first five iterations.

In [22]:
fizzes = itertools.cycle(['', '', 'fizz'])
buzzes = itertools.cycle(['', '', '', '', 'buzz'])
numbers = itertools.count(1)

i = 0
for fizz, buzz, n in zip(fizzes, buzzes, numbers):
    if i < 5:
        print(f'fizz + buzz: {fizz + buzz}, str(n): {str(n)}',
              f'(fizz + buzz) or str(n): {(fizz + buzz) or str(n)}')
        i += 1
    else:
        break


fizz + buzz: , str(n): 1 (fizz + buzz) or str(n): 1
fizz + buzz: , str(n): 2 (fizz + buzz) or str(n): 2
fizz + buzz: fizz, str(n): 3 (fizz + buzz) or str(n): fizz
fizz + buzz: , str(n): 4 (fizz + buzz) or str(n): 4
fizz + buzz: buzz, str(n): 5 (fizz + buzz) or str(n): buzz


The idea is that when `n` is divisible by 3, the `fizzes` will return `'fizz'`. Otherwise it returns an empty string. Simialry, when `n` is divisible by 5, the `buzzes` will return `'buzz'`. Otherwise it returns an empty string. Thus, when `n` is divisible by 3 or 5, or both, the function will return `'fizz'`, `'buzz'`, or `'fizzbuzz'` accordingly. Otherwise, it returns `str(n)`. For how the `or` plays in this, please review what we discussed for the solution "cycle of 15" above.

One more thing. Instead of writing in like a for loop above, the code in the solution actually uses a syntax very similar to list comprehension. 
```python
fizz_buzzes_iter = ((fizz + buzz) or str(n)
                    for fizz, buzz, n in zip(fizzes, buzzes, numbers))
```
But instead of using a list, it puts the expression into a generator. (Notice we use `()` instead of `[]`). We will dive into details about generator here. But in short, generator allowes us to access thoe values from those indifinite iterabels with the `next` statement without running out of memory. Let's see a quick example.

In [23]:
example_generator = ([i, j] for i, j in zip(itertools.count(2), itertools.cycle(['a', 'b'])))
[next(example_generator) for _ in range(5)]

[[2, 'a'], [3, 'b'], [4, 'a'], [5, 'b'], [6, 'a']]

So this concludes our explanation. Let's see them in action.

In [24]:
fizzes = itertools.cycle(['', '', 'fizz'])
buzzes = itertools.cycle(['', '', '', '', 'buzz'])
numbers = itertools.count(1)

fizz_buzzes_iter = ((fizz + buzz) or str(n)
                    for fizz, buzz, n in zip(fizzes, buzzes, numbers))

[next(fizz_buzzes_iter) for _ in range(5)]

['1', '2', 'fizz', '4', 'buzz']

## Random Guessing

In [25]:
def fizz_buzzes_random() -> Iterator[str]:
    counts = [itertools.count(1)] * 15
    for group in zip(*counts):
        random.seed(23_977_775)
        for n in group:
            # Just pick at random
            yield random.choice(
                ['fizzbuzz', 'fizz', str(n), 'buzz']
            )

fb = fizz_buzzes_random()
output_random = [next(fb) for _ in range(20)]

In [26]:
test_fizzbuzz_20(output_random)

Tess Passed


## Matrix Multiplication

In [27]:
def fizz_buzz_mtx(n: int) -> str:
    w = np.array([[1, 0, 0], [2, -2, 0], [2, 0, -2], [3, -3, -3]])
    v = np.array([1, n % 3, n % 5])
    return [str(n), 'fizz', 'buzz', 'fizzbuzz'][np.argmax(w @ v)]

In [28]:
output_mtx = [fizz_buzz_mtx(i) for i in range(1, 21)]
test_fizzbuzz_20(output_mtx)

Tess Passed


Let's write this matrix multiplication code into a mathematical formula.

$$
\begin{bmatrix}
1 &  0 & 0 \\
2 & -2 & 0 \\
2 &  0 & -2 \\
3 & -3 & -3 \\
\end{bmatrix} \times

\begin{bmatrix}
1  & n \% 3 & n \% 5
\end{bmatrix}
=
\begin{bmatrix}
1  \\
2 -  2 (n \% 3) \\
2 -  2 (n \% 5) \\ 
3 - 3(n \% 3) - 3 (n \% 5)
\end{bmatrix}
$$

Now let's define the resulting vector in a function with respect to n and see some of its outputs.

In [29]:
def generate_mtx_output(n: int) -> np.ndarray:
    result = [1, 2 - 2 * (n % 3), 2 - 2 * (n % 5), 3 - 3 * (n % 3) - 3 * (n % 5)]
    return np.array(result)


In [30]:
for n in [3, 5, 7, 15]:
    result = generate_mtx_output(n)
    print(result)

[ 1  2 -4 -6]
[ 1 -2  2 -3]
[ 1  0 -2 -6]
[1 2 2 3]


Notice that when n is divisible by 3 but not divisible by 5, the second element will be the largest in the resulting array. When n is divisible by 5 but not divisible by 3, the 3rd element will be the largest. When n is divisible by both 3 and 5, the 4th element will be the largest. When n is not divisible by both 3 and 5, the first element will be the largest. We will write a formal proof for n is only divisible by 3. You should be able to develop similar proofs for others easily.

Consider a positive integer n that is divisible by 3 but not divisible by 5. Then we have n % 3 = 0 and n \% 5 $\geq$ 1. Then, 2 - 2 * (n % 3) = 2. 2 - 2 * (n % 5) <=0 and 3 - 3(n % 3) - 3 (n % 5) <=0. Obviously, 2 > 1 > 0. Thus, when n is divisible by 3 but not by 5, the 2nd element in the array above will be the largest.

Now let's consider the function `np.argmax`. This function simply returns which index has the largest element in a numpy array. Consider the following example. Notice the third element 2 is the largest in the array.

In [31]:
np.argmax(np.array([1, -2, 2, 3]))

3

With this information, we can extend our for loop above to see how this solution works.

In [32]:
for n in [3, 5, 7, 15]:
    result = generate_mtx_output(n)
    max_index = np.argmax(result)
    output = [str(n), 'fizz', 'buzz', 'fizzbuzz'][max_index]
    print(result, 'max index:', max_index, 'output:', output)

[ 1  2 -4 -6] max index: 1 output: fizz
[ 1 -2  2 -3] max index: 2 output: buzz
[ 1  0 -2 -6] max index: 0 output: 7
[1 2 2 3] max index: 3 output: fizzbuzz


The idea is that when n is divisible by 3 but not by 5, the max index will always be 1 and thus return `'fizz'`. Similar ideas apply to other situations. One interesting question is that how do we come up with the matrix `np.array([[1, 0, 0], [2, -2, 0], [2, 0, -2], [3, -3, -3]])` for the multiplication. The author discusses his thought process in the book. But we can also try another approach - using machine learning to automatically find those weights.

## Machine Learning

In [33]:
def fizz_buzz_class(n: int) -> int:
    return [1, 3, 5, 15].index(math.gcd(n, 15))


class Instance(NamedTuple):
    n: int
    features: torch.Tensor
    label: int

    @staticmethod
    def create(n: int, featurize: Callable) -> 'Instance':
        return Instance(n, featurize(n), fizz_buzz_class(n))


def evaluate(model: torch.nn.Module, 
             data: list,
             verbose = False) -> int:
    num_correct = 0

    # Don't compute gradients when evaluating
    with torch.no_grad():
        for n, features, label in data:
            predicted = torch.argmax(model(features)).item()
            num_correct += predicted == label

            if verbose:
                check = "✓" if predicted == label else "×"
                outputs = [str(n), 'fizz', 'buzz', 'fizzbuzz']
                print(check, n, outputs[predicted], outputs[label])

    return num_correct


def binary_digits(n: int, num_digits: int = 10) -> torch.Tensor:
    digits = []
    for _ in range(num_digits):
        # Need to use floats
        digits.append(float(n % 2))
        n = n // 2
    return torch.tensor(digits)


training_data = [Instance.create(n, binary_digits)
                 for n in range(101, 1023)]

test_data = [Instance.create(n, binary_digits)
             for n in range(1, 101)]

input_dim = len(training_data[0].features)


def train_model(
    hidden_dim: int = 25,
    num_epochs: int = 2500,
    seed: int = 12,
    output_path: str = None
) -> torch.nn.Module:
    torch.manual_seed(seed)

    model = torch.nn.Sequential(
        # Linear layer: input_dim -> hidden_dim
        torch.nn.Linear(in_features=input_dim, out_features=hidden_dim),
        # ReLU(x) = max(x, 0)
        torch.nn.ReLU(),
        # Linear layer: hidden_dim -> 4
        torch.nn.Linear(in_features=hidden_dim, out_features=4)
    )

    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters())

    for epoch in range(num_epochs):
        epoch_loss = 0.0

        for batch in DataLoader(training_data, 
                                batch_size=5, 
                                shuffle=True):
            optimizer.zero_grad()

            predictions = model(batch.features)
            error = loss(predictions, batch.label)
            error.backward()
            epoch_loss += error.item()
            optimizer.step()

        num_correct = evaluate(model, 
                               test_data, 
                               verbose=epoch % 100 == 0)
        print(f"epoch: {epoch:>5} "
            f"accuracy: {num_correct}/100 "
            f"loss: {epoch_loss:.2f}")

    evaluate(model, test_data, verbose=True)

    return model
